In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import os

## Model Layer deflerations:

In [3]:
def weights_variables(shape):
    weight = tf.Variable(tf.random.truncated_normal(shape,stddev=0.1))
    return weight
    
def bias_variables(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W,s):
    return tf.nn.conv2d(input=x,filters=W,strides=[1,1,2,1],padding='SAME') 
    
def max_pool_1x2(x):
    return tf.nn.max_pool2d(input=x,ksize=[1,1,2,1],strides=[1,1,2,1],padding='SAME')    

## Model Creation:

In [4]:
def create_reset_tf1_model():
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.reset_default_graph()

    xs= tf.compat.v1.placeholder(tf.float32,[None,881],name='xs') 
    ys= tf.compat.v1.placeholder(tf.float32,[None,2], name='ys')
    keep_prob = tf.compat.v1.placeholder(tf.float32,name='keep_prob')
    x_image = tf.reshape(xs,[-1,1,881,1])

    ##conv1 layer##
    W_conv1 = weights_variables([1,5,1,32])
    b_conv1 = bias_variables([32])
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1,1) + b_conv1)
    h_pool1 = max_pool_1x2(h_conv1)                        
    h_W1_drop = tf.nn.dropout(h_pool1,1 - (keep_prob))

    ##conv2 layer##
    W_conv2 = weights_variables([1,5,32,64])
    b_conv2 = bias_variables([64])
    h_conv2 = tf.nn.relu(conv2d(h_W1_drop,W_conv2,1) + b_conv2)
    h_pool2 = max_pool_1x2(h_conv2)                       
    h_W2_drop = tf.nn.dropout(h_pool2,1 - (keep_prob))

    ##func1 layer##
    W_fc1 = weights_variables([1*56*64,1024])
    b_fc1 = bias_variables([1024])
    h_pool2_flat = tf.reshape(h_W2_drop,[-1,1*56*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,1 - (keep_prob))

    ##func2 layer##
    W_fc2 = weights_variables([1024,2])
    b_fc2 = bias_variables([2])

    pred = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    prediction = tf.nn.softmax(pred, name= 'prediction')
    return xs, ys, keep_prob, prediction

## Variables and Input Data:

In [21]:
path = u'/home/raayaiardakani.m/workdir/RAMAN/Data/'
model_path = path + 'model/compoent_%03d/compoent.ckpt'
output_spectrum_path = path + 'spectrum.npy'
output_component_path = path + 'multi-component.npy'
data = np.array([np.load(path + str(i) + "component.npy") for i in [0, 7]])
data = data.reshape(data.shape[0] * data.shape[1], data.shape[2])
print("Data shape: ", data.shape)
scaler = preprocessing.StandardScaler().fit(data)
data = scaler.transform(data)
# Bogus labels to feeed the network
label = np.array([(0, 1) for _ in range(40000)])
print("Label Shape: ", label.shape)

Data shape:  (40000, 881)
Label Shape:  (40000, 2)


In [12]:
all_predictions = []
for i in range(167):
    xs, ys, keep_prob, prediction = create_reset_tf1_model()
    saver = tf.compat.v1.train.Saver()
    with tf.compat.v1.Session() as sess:
        model_file = model_path % i
        # Restore variables from disk.
        saver.restore(sess, model_file)
        print("Model restored.")
        # Check the values of the variables
        valid_ypred = sess.run(prediction,feed_dict={ xs: data, ys: label, keep_prob : 1.0})
        all_predictions.append(np.argmax(valid_ypred, 1))

INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_000/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_001/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_002/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_003/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_004/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_005/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/workdir/RAMAN/Data/model/compoent_006/compoent.ckpt
Model restored.
INFO:tensorflow:Restoring parameters from /home/raayaiardakani.m/work

## Save the predictions and aggregated data for future use:

In [22]:
all_predictions = np.array(all_predictions)
print(all_predictions.shape)
np.save(output_spectrum_path, data)
np.save(output_component_path, all_predictions)

(167, 40000)
